In [28]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np

pd.set_option('plotting.backend', 'plotly')
from datetime import date, datetime as dt
import os, re
import plotly.colors
import plotly.graph_objects as go, plotly.express as px, plotly.figure_factory as ff
from plotly.offline import init_notebook_mode
from plotly.subplots import make_subplots


from tennis_utils.player import TennisDataLoader, TennisPlayerDataLoader, TennisPlayer


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [21]:

player_name = 'Roger Federer'
data_path = os.getcwd()+'/data'

tdl = TennisDataLoader(data_path+'/matches.parquet', data_path+'/players.parquet')
matches_df, players_df = tdl.matches, tdl.players

tpdl = TennisPlayerDataLoader(player_name, matches_df, players_df)

# Subset selected player matches data
player_matches = tpdl.player_matches #matches_df[matches_df['player_name'] == player_name]
player_details = tpdl.player_details #players_df[players_df['player_name']==player_name]
player_rank = tpdl.player_rank

tp = TennisPlayer(player_name, player_matches, player_rank, player_details)

Index(['Lost', 'Won'], dtype='object')

In [32]:
px.colors.sequential.Rainbow

['rgb(150,0,90)',
 'rgb(0,0,200)',
 'rgb(0,25,255)',
 'rgb(0,152,255)',
 'rgb(44,255,150)',
 'rgb(151,255,0)',
 'rgb(255,234,0)',
 'rgb(255,111,0)',
 'rgb(255,0,0)']

In [3]:
#df = tp.get_stats()
df = tp.selected_matches
df['percServeWon'] = (df['perc1stIn'] * df['perc1stWon']) + ((1-df['perc1stIn']) * df['perc2ndWon'])

   
def get_stats(df):

    t = (df.groupby('year')
            .agg(mean_perc1stIn = ('perc1stIn', np.mean),
                mean_perc1stWon = ('perc1stWon', np.mean),
                mean_perc2ndWon = ('perc2ndWon', np.mean),
                mean_percReturnWon = ('percReturnWon', np.mean),
                mean_percServeWon = ('percServeWon', np.mean),
                std_perc1stIn = ('perc1stIn', np.std),
                std_perc1stWon = ('perc1stWon', np.std),
                std_perc2ndWon = ('perc2ndWon', np.std),
                std_percReturnWon = ('percReturnWon', np.std),
                std_percServeWon = ('percServeWon', np.std))
            .reset_index()
    )
    return t

t = get_stats(df)

In [33]:
# df = (df.groupby(['year', 'quarter'])
#     .agg(mean_perc1stIn = ('perc1stIn', np.mean),
#             mean_perc1stWon = ('perc1stWon', np.mean),
#             mean_perc2ndWon = ('perc2ndWon', np.mean),
#             mean_percReturnWon = ('percReturnWon', np.mean),
#             std_perc1stIn = ('perc1stIn', np.std),
#             std_perc1stWon = ('perc1stWon', np.std),
#             std_perc2ndWon = ('perc2ndWon', np.std),
#             std_percReturnWon = ('percReturnWon', np.std))
#      .reset_index()
#      .assign(idx = lambda x: x['year'].astype(str) + ' Q' + x['quarter'].astype(str))
# )
# df

In [44]:
def plot_stats_by_year(df, c):

    assert 'mean_' + c in df.columns and 'std_' + c in df.columns

    
    x, m, s = df['year'], df['mean_'+c].to_numpy(), df['std_'+c].to_numpy()
    y1, y2 = m + 2*s, m-2*s


    fig = go.Figure()

    

    fig.add_trace(
            go.Scatter(
                x=x, y=y1,
                name='Upper Band',
                fill=None,
                mode='lines',
                line=dict(color='darksalmon', width=1)
        ),
    )
    
    fig.add_trace(
            go.Scatter(
                x=x, y=y2,
                name='Lower Band',
                fill='tonexty', # fill area between trace0 and trace1
                mode='lines',
                line=dict(color='darksalmon', width=1)),
    )

    fig.add_trace(
        go.Scatter(
            x=x, y=m,
            name=f'Mean {c}',
            mode='lines+markers',
            marker={'color': 'mediumblue'}
        )
    )

    fig.update_layout(
        yaxis={'range':[np.min(y2) -0.1, np.max(y1)+0.1]}
    )
    return fig

In [45]:
c = 'percServeWon'

plot_stats_by_year(t, c)




In [46]:
c = 'percReturnWon'

plot_stats_by_year(t, c)